In [1]:
import glob
import os
import shutil
from PIL import Image 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nibabel as nib

from modules.scandata import MriScan, MriSlice, TumourSegmentation, ScanType, ScanPlane

In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
#import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2022-11-22 10:16:53.836976: I tensorflow/core/tpu/tpu_api_dlsym_initializer.cc:116] Libtpu path is: libtpu.so


In [3]:
tf.config.list_logical_devices('TPU')

2022-11-22 10:16:58.255788: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[LogicalDevice(name='/device:TPU:0', device_type='TPU'),
 LogicalDevice(name='/device:TPU:1', device_type='TPU'),
 LogicalDevice(name='/device:TPU:2', device_type='TPU'),
 LogicalDevice(name='/device:TPU:3', device_type='TPU'),
 LogicalDevice(name='/device:TPU:4', device_type='TPU'),
 LogicalDevice(name='/device:TPU:5', device_type='TPU'),
 LogicalDevice(name='/device:TPU:6', device_type='TPU'),
 LogicalDevice(name='/device:TPU:7', device_type='TPU')]

2022-11-22 10:17:04.409291: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x7030840 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
2022-11-22 10:17:04.409350: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): TPU, 2a886c8
2022-11-22 10:17:04.409356: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (1): TPU, 2a886c8
2022-11-22 10:17:04.409361: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (2): TPU, 2a886c8
2022-11-22 10:17:04.409366: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (3): TPU, 2a886c8
2022-11-22 10:17:04.409371: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (4): TPU, 2a886c8
2022-11-22 10:17:04.409376: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (5): TPU, 2a886c8
2022-11-22 10:17:04.409381: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecuto

In [4]:
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.config.experimental_connect_to_cluster(cluster_resolver)
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
strategy = tf.distribute.TPUStrategy(cluster_resolver)


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU

In [5]:
batch_size = 64
img_height = 240
img_width = 240
data_dir = os.path.join('data','UPENN-GBM','slice_classification','train')


In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    color_mode="rgba",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 49725 files belonging to 8 classes.
Using 39780 files for training.


In [7]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [8]:
train_ds.map()

TypeError: map() missing 1 required positional argument: 'map_func'

In [9]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    color_mode="rgba",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 49725 files belonging to 8 classes.
Using 9945 files for validation.


In [10]:
class_names = train_ds.class_names
print(class_names)

['background', 'background_contrast', 'background_edema', 'background_edema_contrast', 'background_tumour', 'background_tumour_contrast', 'background_tumour_edema', 'background_tumour_edema_contrast']


In [11]:
# Class weights to account for class imbalance
class_weight = {
    0: 0.35709557053442464, 
    1: 139.94103773584905, 
    2: 0.539409090909091, 
    3: 4.525244051250763, 
    4: 3708.4375, 
    5: 412.0486111111111, 
    6: 21.943417159763314, 
    7: 0.3258016692290797
}

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [12]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [13]:
normalization_layer = layers.Rescaling(1./(2**8-1))

In [14]:
num_classes = len(class_names)

In [ ]:

with strategy.scope():
    model = Sequential([
      layers.Rescaling(1./(2**8-1), input_shape=(img_height, img_width, 4)),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(num_classes)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  class_weight=class_weight
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  4)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)



In [ ]:

plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    #with tf.device('/CPU:0'):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
tf.config.list_physical_devices()

In [17]:
with strategy.scope():
    model = Sequential([
      #data_augmentation,
      layers.Rescaling(1./(2**8-1), input_shape=(img_height,
                                  img_width,
                                  4)),
      layers.Conv2D(8, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(128, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dropout(0.2),
      layers.Dense(128, activation='relu'),
      layers.Dropout(0.2),
      layers.Dense(128, activation='relu'),
      layers.Dropout(0.2),
      layers.Dense(num_classes, name="outputs")
    ])

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

2022-11-22 10:19:58.567593: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:57] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. RandomUniform


In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 240, 240, 4)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 240, 240, 8)       296       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 120, 120, 8)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 120, 120, 16)      1168      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 60, 60, 16)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 60, 60, 32)       

In [19]:
epochs = 80
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  class_weight=class_weight
)

2022-11-22 10:20:10.220530: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 39780
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_STRING
        }
      }
    }
  }
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
     

Epoch 1/80


2022-11-22 10:20:13.711926: I tensorflow/core/tpu/graph_rewrite/encapsulate_tpu_computations_pass.cc:263] Subgraph fingerprint:7726921516384453180
2022-11-22 10:20:13.822042: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:828] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/Adam/AssignAddVariableOp.
2022-11-22 10:20:14.026399: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:828] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/Adam/AssignAddVariableOp.
2022-11-22 10:20:18.071889: I tensorflow/core/tpu/kernels/tpu_compilation_cache_interface.cc:435] TPU host compilation cache miss: cache_key(4559137277479312852), session_name()


  3/622 [..............................] - ETA: 39s - loss: 1.7150 - accuracy: 0.1094   

2022-11-22 10:20:22.219394: I tensorflow/core/tpu/kernels/tpu_compile_op_common.cc:180] Compilation of 4559137277479312852 with session name  took 4.147408049s and succeeded
2022-11-22 10:20:22.229070: I tensorflow/core/tpu/kernels/tpu_compilation_cache_interface.cc:468] TPU host compilation cache: compilation complete for cache_key(4559137277479312852), session_name(), subgraph_key(std::string(property.function_name) = "cluster_train_function_7726921516384453180", property.function_library_fingerprint = 13149553722443770526, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, topology.MissingChipCount() = 0, std::string(property.shapes_prefix) = "8,240,240,4,;8,;8,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
2022-11-22 10:20:22.229126: I tensorflow

...

621/622 [============================>.] - ETA: 0s - loss: 1.8413 - accuracy: 0.3113

2022-11-22 10:20:52.356741: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 9945
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:7"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_STRING
        }
      }
    }
  }
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      

622/622 [==============================] - 53s 68ms/step - loss: 1.8399 - accuracy: 0.3109 - val_loss: 1.9697 - val_accuracy: 0.0012


2022-11-22 10:21:04.251919: I tensorflow/core/tpu/kernels/tpu_compile_op_common.cc:180] Compilation of 12200965989362535932 with session name  took 2.210454818s and succeeded
2022-11-22 10:21:04.254745: I tensorflow/core/tpu/kernels/tpu_compilation_cache_interface.cc:468] TPU host compilation cache: compilation complete for cache_key(12200965989362535932), session_name(), subgraph_key(std::string(property.function_name) = "cluster_test_function_9790963054067299412", property.function_library_fingerprint = 12869646956898360300, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, topology.MissingChipCount() = 0, std::string(property.shapes_prefix) = "4,240,240,4,;4,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
2022-11-22 10:21:04.254782: I tensorflow/c

Epoch 2/80
622/622 [==============================] - 31s 50ms/step - loss: 1.7480 - accuracy: 0.1308 - val_loss: 1.9296 - val_accuracy: 0.0315
Epoch 3/80
622/622 [==============================] - 31s 49ms/step - loss: 1.7267 - accuracy: 0.0966 - val_loss: 1.9297 - val_accuracy: 0.2452
Epoch 4/80
622/622 [==============================] - 30s 49ms/step - loss: 1.7178 - accuracy: 0.1281 - val_loss: 1.9909 - val_accuracy: 0.0012
Epoch 5/80
622/622 [==============================] - 30s 48ms/step - loss: 1.7142 - accuracy: 0.1111 - val_loss: 1.9785 - val_accuracy: 6.0332e-04
Epoch 6/80
622/622 [==============================] - 30s 49ms/step - loss: 1.7398 - accuracy: 0.0794 - val_loss: 1.9730 - val_accuracy: 0.0012
Epoch 7/80
622/622 [==============================] - 31s 49ms/step - loss: 1.7831 - accuracy: 0.0974 - val_loss: 1.9349 - val_accuracy: 0.2444
Epoch 8/80
622/622 [==============================] - 30s 48ms/step - loss: 1.7155 - accuracy: 0.1151 - val_loss: 1.9887 - val_accur

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
history